In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch_lightning
!pip install transformers
!pip install dataset

     |████████████████████████████████| 582 kB 5.1 MB/s 
     |████████████████████████████████| 398 kB 66.2 MB/s 
     |████████████████████████████████| 136 kB 75.4 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 1.1 MB 55.1 MB/s 
     |████████████████████████████████| 144 kB 56.3 MB/s 
     |████████████████████████████████| 271 kB 74.2 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [ ]:
batchsize=10
import pandas as pd 
import numpy as np
import re
import os
import math
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from torch.nn import BCEWithLogitsLoss,MSELoss,BCELoss
from transformers import AdamW, ElectraTokenizer, ElectraModel, ElectraConfig

%matplotlib inline

In [ ]:
#Define the computation ressources
device =torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

In [ ]:
import pickle
with open('./drive/MyDrive/Data/English_TGD2_TGD_All_data.pkl','rb') as file:
    TGD_All_data=pickle.load(file)


In [ ]:
# Remove the movie tokens from the topic field as they do not help for predicting the topic of the utterance
def remove_movie_from_path_goal(TGD_All_data,typee='train'):
  for i, dial in enumerate(TGD_All_data[typee]):
    goal_path=dial['goal_path']
    for k, v in goal_path.items():
      v=[x for x in v if x[0]!='@']
      goal_path[k]=v
    
    TGD_All_data[typee][i]['goal_path']=goal_path
  return TGD_All_data[typee]

TGD_All_data['train']=remove_movie_from_path_goal(TGD_All_data,typee='train')
TGD_All_data['test']=remove_movie_from_path_goal(TGD_All_data,typee='test')
TGD_All_data['valid']=remove_movie_from_path_goal(TGD_All_data,typee='valid')

In [ ]:
# reformulate our data by combining both intent and topics topics
def create_entent_topic(TGD_All_data=TGD_All_data,typee="train"):

  topic_list=TGD_All_data["real_topics"].values()
  for i,dial in enumerate(TGD_All_data[typee]):
    goal_path=dial["goal_path"]
    new_goal_path={}
    
    for k,v in goal_path.items():
      topics=[x for x in v if x in topic_list]
      intents=[x for x in v if x not in topic_list]
      if(len(topics)==0):
        topics=["no-topic"]
      new_goal_path[k]={"topics":intents+topics}
    TGD_All_data[typee][i]["goal_path"]=new_goal_path

  return TGD_All_data
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="train")
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="test")
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="valid")





In [ ]:
# Create our training data where we combine each utterance with the previous ones and the previous topics.
# the current topic is defined as a label for the new formulated row
import re
def create_path_included_data(TGD_All_data,typee='valid'):
  data_dialogue_rows=[]
  data_dialogue_labels=[]
  for i,dial in enumerate(TGD_All_data[typee]):
    goal_path=dial['goal_path']
    messages=dial['messages']

    line=' '
    for i,v in enumerate(messages):

      m='<'+v['role'].lower()+'>'+' '+v['content']
      label=' '.join(x for x in goal_path[v['local_id']]["topics"])
      
      
      m=re.sub(' +', ' ', m)
      line=line+m
    
      line=re.sub(' +', ' ', line).strip()
      data_dialogue_labels.append(label)
      data_dialogue_rows.append(line)
      line=line+' <topic> '+ label +" </topic> "
    
  return data_dialogue_rows,data_dialogue_labels
    

train_rows,train_labels=create_path_included_data(TGD_All_data,typee='train')   
test_rows,test_labels=create_path_included_data(TGD_All_data,typee='test')   
valid_rows,valid_labels=create_path_included_data(TGD_All_data,typee='valid')   
    

In [ ]:
# Mask the movie tokens from the utterance to help the model predicting the topic of the utterance independently from the mentioned movie

def add_movie_mask( train_rows):
  t=[]
  for m in train_rows:
    if("@" in m ):
        lin= np.array([word for word in m.split(" ") ])
        indexes=[i for i,v in enumerate(lin) if v[0]=="@"]
        lin[indexes]="[Movie]" 
        m=" ".join(p for p in lin)
        m=re.sub(' +', ' ', m).strip()
    t.append(m)
  return t
train_rows=add_movie_mask( train_rows)
test_rows=add_movie_mask( test_rows)
valid_rows=add_movie_mask( valid_rows)

      

In [ ]:
#an example of the training rows
train_rows[:20]

['<recommender> how are things',
 "<recommender> how are things <topic> start conversation no-topic </topic> <seeker> it's okay recently , still alone , struggling in shanghai , checking in every day , nothing else . how are you ?",
 "<recommender> how are things <topic> start conversation no-topic </topic> <seeker> it's okay recently , still alone , struggling in shanghai , checking in every day , nothing else . how are you ? <topic> talk struggle </topic> <recommender> i've been doing well recently , and the local tyrant will receive it next week , which is the company's reward .",
 "<recommender> how are things <topic> start conversation no-topic </topic> <seeker> it's okay recently , still alone , struggling in shanghai , checking in every day , nothing else . how are you ? <topic> talk struggle </topic> <recommender> i've been doing well recently , and the local tyrant will receive it next week , which is the company's reward . <topic> talk award </topic> <seeker> it's really outs

In [ ]:
#get the unique topic tokens so that we can add them to the vocabulary of the pre-trained model
topic_tokens=[]
for i,dial in enumerate(TGD_All_data["train"]):
    goal_path=dial["goal_path"]    
    for k,v in goal_path.items():
      
      topic_tokens=topic_tokens+[x for x in v["topics"]]
topic_tokens=np.unique(topic_tokens)
topic_list=list(TGD_All_data['real_topics'].values())
topic_list=list(np.unique(topic_list+list(topic_tokens)))

In [ ]:
#Instantiate the pre-trained electra tokenizer small version
model_name="google/electra-small-discriminator"

tokenizer_4rec = ElectraTokenizer.from_pretrained(model_name, do_lower_case=True)



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
#define our data subsets 
train_data=train_rows
test_data=test_rows
valid_data=valid_rows

In [ ]:
# transform the topic list into list of words
topic_list=" ".join(x for x in topic_list)
topic_list=list(np.unique(topic_list.split())) 

In [ ]:
#verify the list of tokens that does not exist in the pre-trained model vocabulary so that we can add them
topics_not_existed=[x for x in topic_list if x not in tokenizer_4rec.get_vocab().keys() ]

In [ ]:
#Define the special tokens that we are gonna add to the utterances

special_tokens=['<entent>','</entent>','<recommender>','<topic>',"</topic>",'<seeker>',"no-topic"]

In [ ]:
#Define the words that we are gonna add to the pre-trained model vocabulary
vocab_to_add=topics_not_existed+special_tokens+['uk']

In [ ]:
#Create one-hot vector representation for the labels
def get_train_label_vector(train_labels, vocab_topics):

  topic_labels_vector=[]

  list_labels=[]
  
  for i, topic in enumerate(train_labels): 
    label_topic=np.zeros(len(vocab_topics))
    for j,t in enumerate(topic.split()): 
        
        label_topic[vocab_topics.index(t)]=1

    topic_labels_vector.append(label_topic)

 
  return topic_labels_vector
  
train_labels_vector=get_train_label_vector(train_labels, topic_list)
test_labels_vector=get_train_label_vector(test_labels, topic_list)
valid_labels_vector=get_train_label_vector(valid_labels, topic_list)


In [ ]:
#Rename the features and labels subsets
X_train, y_train_topic_vector =train_rows,train_labels_vector
X_test, y_test_topic_vector =test_rows,test_labels_vector
X_valid, y_valid_topic_vector =valid_rows,valid_labels_vector

#Design the model

In [ ]:
#Instentiate the pre-trained electra model
pre_tained_model=ElectraModel.from_pretrained('google/electra-small-discriminator')


Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer_4rec.add_tokens(vocab_to_add) # add the pre-defined vocab to add to the electra tokenizer and pre-trained model
pre_tained_model.resize_token_embeddings(len(tokenizer_4rec)) 

Embedding(31238, 128)

In [ ]:
# rename our sub sets 
train_data= X_train
test_data = X_test
valid_data = X_valid



In [ ]:
#Define a tokenization function using the pre-trained tokenizer model version
def tokenize_inputs(liste, tokenizer, num_embeddings=512):
    
    outputs = tokenizer.batch_encode_plus(liste,truncation=True,return_tensors='pt',return_attention_mask=True,max_length=512, padding='max_length')
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids =outputs.input_ids
    

    return (input_ids,outputs.attention_mask)
    

In [ ]:
#Tokenize train, test, and validation subsets
train_input_ids,train_attention_masks=tokenize_inputs(train_data, tokenizer_4rec, num_embeddings=512) #1024
valid_input_ids,valid_attention_masks=tokenize_inputs(valid_data, tokenizer_4rec, num_embeddings=512)

In [ ]:
test_input_ids,test_attention_masks=tokenize_inputs(test_data, tokenizer_4rec, num_embeddings=512)


In [ ]:
#rename our data names
X_train,train_masks,Y_train_topic= (train_input_ids,train_attention_masks,torch.tensor(np.array(y_train_topic_vector),dtype=torch.float32))

X_valid,valid_masks,Y_valid_topic= (valid_input_ids,valid_attention_masks,torch.tensor(np.array(y_valid_topic_vector),dtype=torch.float32))

In [ ]:
#Make the data convininet to be used by the model in the pytprch framework

train_data = TensorDataset(X_train, train_masks, Y_train_topic)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,\
                              sampler=train_sampler,\
                              batch_size=batchsize)

validation_data = TensorDataset(X_valid, valid_masks, Y_valid_topic)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,\
                                   sampler=validation_sampler,\
                                   batch_size=batchsize)

In [ ]:
#CREATE THE CLASSIFICATION MODEL
class ElectraForMultiLabelSequenceClassification(torch.nn.Module):
  
  def __init__(self, pre_trained_model,num_labels_topics=2,num_labels_entent=2):
    super(ElectraForMultiLabelSequenceClassification, self).__init__()
    self.num_labels_topics = num_labels_topics
    self.electra = pre_trained_model#Use the pretrqined model
    self.layer1 = torch.nn.Linear(256, 1000)# add a linear layer on top of the encoding  discriminator layers of Electra 
    self.layer2  = torch.nn.Linear(1000, 2000)# add a linear layer on top of the encoding  discriminator layers of Electra 
    self.layer3  = torch.nn.Linear(2000, 3000)#change it to 5000 # add a linear layer on top of the encoding  discriminator layers of Electra 
    self.classifier_topics = torch.nn.Linear(3000, self.num_labels_topics)# add a linear layer on top of the encoding  discriminator layers of Electra 
    self.drop=torch.nn.Dropout(p=0.4)
    self.leaky=torch.nn.ReLU()

    torch.nn.init.xavier_normal_(self.classifier_topics.weight) #initialise the weight of th linear layer
    torch.nn.init.xavier_normal_(self.layer1.weight) #initialise the weight of th linear layer
    torch.nn.init.xavier_normal_(self.layer2.weight) #initialise the weight of th linear layer
    torch.nn.init.xavier_normal_(self.layer3.weight) #initialise the weight of th linear layer

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels_topic=None,labels_entent=None):
    # last hidden layer
    last_hidden_state = self.electra(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
    # pool the outputs into a mean vector
    mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
    l1 = self.leaky(self.layer1(mean_last_hidden_state))
    l1=self.drop(l1)
    l2 = self.leaky(self.layer2(l1))
    l3 = self.leaky(self.layer3(l2))
    l3=self.drop(l3)
    logits_topics = torch.sigmoid(self.classifier_topics(l3))
        
    if labels_topic is not None:
      loss_fct_topic = BCELoss()
      loss1 = loss_fct_topic(logits_topics.view(-1, self.num_labels_topics),\
                      labels_topic.view(-1, self.num_labels_topics))
      
      
      return loss1
    else:
      return logits_topics

#This code borrow ideas from  :https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state
    
  

# len(Y_train[0]) = 6


In [ ]:
#Instantiate the learner model and define that it will be runing on the pre-defined device
model = ElectraForMultiLabelSequenceClassification(pre_tained_model,num_labels_topics=len(Y_train_topic[0]))
model.to(device)

ElectraForMultiLabelSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(31238, 128)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featu

In [ ]:
#choose the learning optimizer algorithm
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01, correct_bias=False)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
#Define the training function ,The training method code is copied from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df
def train(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          model_save_path,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
  """
  Train the model and save the model with the lowest validation loss
  """

  model.to(device)

  for i in trange(num_epochs, desc="Epoch"):
    
    actual_epoch = start_epoch + i

    

    model.train()

    tr_loss = 0
    num_train_samples = 0

    for step, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_topics = batch
      # Clear out the gradients (by default they accumulate)
      optimizer.zero_grad()
      # Forward pass
      loss = model(b_input_ids, attention_mask=b_input_mask, labels_topic=b_labels_topics)
      # store train loss
      tr_loss += loss.item()
      num_train_samples += b_labels_topics.size(0)
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      #scheduler.step()

    # Update tracking variables
    epoch_train_loss = tr_loss/num_train_samples
    train_loss_set.append(epoch_train_loss)

    print("Train loss: {}".format(epoch_train_loss))

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss = 0
    num_eval_samples = 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_topics = batch
      # Telling the model not to compute or store gradients,
      # saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate validation loss
        loss= model(b_input_ids, attention_mask=b_input_mask, labels_topic=b_labels_topics)
        # store valid loss
        eval_loss += loss.item()
        num_eval_samples += b_labels_topics.size(0)

    epoch_eval_loss = eval_loss/num_eval_samples
    valid_loss_set.append(epoch_eval_loss)

    print("Valid loss: {}".format(epoch_eval_loss))
    
    if lowest_eval_loss == None:
      lowest_eval_loss = epoch_eval_loss
      # save model
      save_model(model, model_save_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)
    else:
      if epoch_eval_loss < lowest_eval_loss:
        lowest_eval_loss = epoch_eval_loss
        # save model
        save_model(model, model_save_path, actual_epoch,\
                   lowest_eval_loss, train_loss_set, valid_loss_set)
    print("\n")

  return model, train_loss_set, valid_loss_set


def save_model(model, save_path, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist):
  """
  Save the model to the path directory provided
  """
  model_to_save = model.module if hasattr(model, 'module') else model
  checkpoint = {'epochs': epochs, \
                'lowest_eval_loss': lowest_eval_loss,\
                'state_dict': model_to_save.state_dict(),\
                'train_loss_hist': train_loss_hist,\
                'valid_loss_hist': valid_loss_hist
               }
  torch.save(checkpoint, save_path)
  print("Saving model at epoch {} with validation loss of {}".format(epochs,\
                                                                     lowest_eval_loss))
  return
  
def load_model(save_path):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = ElectraForMultiLabelSequenceClassification(num_labels=model_state_dict["classifier.weight"].size()[0])
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist

In [ ]:
# Define the saving path
import os
num_epochs=30

cwd = os.getcwd()
model_save_path = output_model_file = os.path.join(cwd, "drive/MyDrive/Models/TopicPrediction/TGD_Classification_Concatenate_Both_Topic_Entent_Sigmoid_2Essay.bin")


In [ ]:
model_save_path = output_model_file = os.path.join(cwd, "drive/MyDrive/Models/TopicPrediction/TGD_Classification_Concatenate_Both_Topic_Entent_Sigmoid.bin")


In [ ]:
#Train the model
model, train_loss_set, valid_loss_set = train(model=model,\
                                              num_epochs=num_epochs,\
                                              optimizer=optimizer,\
                                              train_dataloader=train_dataloader,\
                                              valid_dataloader=validation_dataloader,\
                                              model_save_path=model_save_path,\
                                              device=device)
#0.016783231785479146

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Train loss: 0.0004201373413772305
Valid loss: 0.00017138624336803928
Saving model at epoch 0 with validation loss of None


Epoch:   3%|▎         | 1/30 [25:56<12:32:27, 1556.80s/it]

Saving model at epoch 0 with validation loss of 0.00017138624336803928


Train loss: 0.00015501507338890023
Valid loss: 0.00012067284470362134
Saving model at epoch 1 with validation loss of 0.00017138624336803928


Epoch:   7%|▋         | 2/30 [51:49<12:05:25, 1554.50s/it]

...

Saving model at epoch 27 with validation loss of 7.664176486992491e-05


Train loss: 2.776455946040376e-05
Valid loss: 7.777313138253448e-05


Epoch:  97%|█████████▋| 29/30 [12:30:39<25:54, 1554.63s/it]

Saving model at epoch 28 with validation loss of 7.664176486992491e-05


Train loss: 2.6867796875626645e-05
Valid loss: 7.779156769456906e-05


Epoch: 100%|██████████| 30/30 [12:56:32<00:00, 1553.07s/it]

Saving model at epoch 29 with validation loss of 7.664176486992491e-05




In [ ]:
#Restore the best version of the trained model
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint = torch.load(model_save_path)
model_state_dict = checkpoint['state_dict']
model = ElectraForMultiLabelSequenceClassification(pre_tained_model,num_labels_topics=model_state_dict["classifier_topics.weight"].size()[0])
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [ ]:
#Create a function to generate tpic prediction from a given text
##This code borrow ideas from : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

def generate_predictions(model, features,mask, num_labels, device="cpu", batch_size=32):
  num_iter = math.ceil(features.shape[0]/batch_size)
  
  pred_probs = np.array([]).reshape(0, num_labels)
  
  model.to(device)
  model.eval()
  
  for i in range(num_iter):
    X = features[i*batch_size:(i+1)*batch_size,:]
    masks = mask[i*batch_size:(i+1)*batch_size,:]
    X = X.to(device)
    masks = masks.to(device)
    with torch.no_grad():
      logits = model(input_ids=X, attention_mask=masks)
      #logits = logits.sigmoid().detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      pred_probs = np.vstack([pred_probs, logits])
  
  return pred_probs
  
num_labels = len(valid_labels_vector[0])
pred_probs = generate_predictions(model, test_input_ids,test_attention_masks, num_labels, device=device, batch_size=32)
prob=np.round(pred_probs)


#Evaluate the results 

In [ ]:
from sklearn.metrics import label_ranking_average_precision_score,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score
label_ranking_average_precision_score(y_test_topic_vector, prob) 
#0.9027679914144469



0.9027679914144469

In [ ]:
accuracy_score(y_test_topic_vector, prob) 
#0.7983176036109971

0.7983176036109971

In [ ]:
roc_auc_score(y_test_topic_vector, prob,average="micro") 
#0.9606383814826487

0.9606383814826487

In [ ]:
precision_score(y_test_topic_vector, prob,average="micro") 
#0.9692994051553205

0.9692994051553205

In [ ]:
precision_score(y_test_topic_vector, prob,average="macro") 
#0.16962602821072634


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.16962602821072634

In [ ]:
recall_score(y_test_topic_vector, prob,average="micro") 
#0.9213154918959668

0.9213154918959668

In [ ]:
f1_score(y_test_topic_vector, prob,average="micro") 
#0.9446985313063644

0.9446985313063644

In [ ]:
#Define hit@n  algorithm 

def hit(data_predicted=pred_probs,data_real=y_test_topic_vector,n=1):

  hit=0


  for i,p in enumerate(y_test_topic_vector):
    real_indexes=[j for j,i in enumerate(p) if i ==1]
    original_topics=" ".join(topic_list[x] for x in real_indexes)
    #print(f" real topics: {original_topics}")
    
    prob=pred_probs[i].argsort()[-n*len(real_indexes):][::-1]
    predicted_topics=" ".join(topic_list[x] for x in prob)
    #print(f" predicted topics: {predicted_topics}")
    check =  all(item in prob for item in real_indexes)
    if (check ==True):
      hit=hit+1
  #print(hit/len(y_test_topic_vector))
  return hit/len(y_test_topic_vector)


In [ ]:
#Compute the Hit@1 results
result=hit(data_predicted=pred_probs,data_real=y_test_topic_vector,n=1)
result
#0.8359663520722199

0.8359663520722199

In [ ]:
#Compute the Hit@3 results

result=hit(data_predicted=pred_probs,data_real=y_test_topic_vector,n=3)
result
#0.9070578580221584

0.9070578580221584

In [ ]:
#Compute the Hit@5 results

result=hit(data_predicted=pred_probs,data_real=y_test_topic_vector,n=5)
result
#0.9191629052113254


0.9191629052113254

In [ ]:
#Compute the Hit@10 results

result=hit(data_predicted=pred_probs,data_real=y_test_topic_vector,n=10)
result
#0.9344480919162905


0.9344480919162905